# Benchmark de Estrategias en Múltiples Datasets

## Objetivo del Notebook

Este notebook es un benchmark de alto nivel. Su objetivo es comparar el rendimiento de tres estrategias fundamentales para el manejo de datos desbalanceados, aplicadas a cuatro datasets de crédito y fraude.

**Estrategias a Comparar:**
1.  **Estrategia 1: Convencional** (Línea Base).
2.  **Estrategia 2: Coste Sensible Fijo** (Usando `class_weight='balanced'`).
3.  **Estrategia 3: Balanceo de Datos** (Usando `SMOTE`).

**Métricas Clave:**
* **`Balanced Accuracy`:** Mide el equilibrio general del modelo.
* **`F1-Score (Clase Minoritaria)`:** Mide la efectividad (equilibrio Precisión/Recall) para detectar la clase "Mal Riesgo" o "Fraude", que es nuestro objetivo principal.

In [1]:
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Modelos a comparar
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline # Renombrado para evitar conflicto

# Métricas de evaluación
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, f1_score

# Para mostrar la tabla final
from IPython.display import display

## 1. Definición de Modelos a Probar

Se seleccionan tres tipos de modelos representativos: Regresión Logística (lineal), Árbol de Decisión (árbol simple) y Random Forest (ensamble de árboles).

**Nota sobre SVC (Support Vector Classifier):**
Aunque `SVC` fue importado en la celda anterior, se ha **comentado y excluido** de este benchmark (`models_to_test`).

La razón es su **alto coste computacional**. El tiempo de entrenamiento de SVC escala de forma cuadrática o cúbica ($O(n^2)$ a $O(n^3)$) con el número de muestras. Ejecutarlo en datasets grandes como "Credit Card Fraud" (284k muestras) o "Give Me Some Credit" (150k muestras) bajo 3 estrategias diferentes (36* ejecuciones) haría que el notebook tardara horas en completarse.

El `SVC` sí se analiza en profundidad en el notebook `deteccion_impago_german_credit_data.ipynb`, que utiliza un dataset más pequeño (1000 muestras) donde su coste es manejable.

In [2]:
models_to_test = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=42),
    #'SVC': SVC(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42)
}

## 2. Función de Evaluación 1: El Benchmark Convencional

La siguiente función, `evaluate_models`, se encarga de establecer la **línea base (benchmark)**. Esta es la referencia "ingenua" contra la cual mediremos todas nuestras estrategias.

**¿Qué hace esta función?**
1.  **Recibe** un dataset (`X`, `y`), un diccionario de modelos (`models`) y un nombre (`dataset_name`).
2.  **Preprocesa** los datos usando un `ColumnTransformer` estándar (escala numéricos, codifica categóricos).
3.  **Divide** los datos en entrenamiento y prueba (70/30) con estratificación.
4.  **Usa un `Pipeline` estándar de scikit-learn**.
5.  **Entrena** cada modelo (`LogisticRegression`, `RandomForest`, etc.) **sin ningún parámetro de coste** (ej. `class_weight=None`). Esto significa que el modelo trata todos los errores por igual.
6.  **Evalúa** el rendimiento en el set de prueba usando `balanced_accuracy` y `f1_score`.
7.  **Devuelve** una lista de resultados que se usará para construir la tabla "Tipo 1: Convencional".

In [3]:
def evaluate_models(X, y, models, dataset_name):
    """
    Entrena y evalúa una lista de modelos en un dataset.
    Devuelve una lista de diccionarios con los resultados de cada modelo.
    """
    print(f"--- Evaluando Dataset: {dataset_name} ---")
    
    results_list = []
    
    # Preprocesamiento
    categorical_features = X.select_dtypes(include=['category', 'object']).columns
    numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
        ],
        remainder='passthrough'
    )
    
    # División de datos
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    
    # Bucle para probar cada modelo
    for model_name, model in models.items():
        print(f"\nProbando modelo: {model_name}...")
        
        # Crear y entrenar el pipeline
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', model)
        ])
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        
        # Calcular y mostrar métricas
        print("Matriz de Confusión:")
        print(confusion_matrix(y_test, y_pred))
        
        bal_acc = balanced_accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, pos_label=1, zero_division=0)
        
        print(f"Balanced Accuracy: {bal_acc:.3f}")
        print(f"F1-Score (clase 'mala'): {f1:.3f}")
        
        # Guardar resultados
        results_list.append({
            'dataset': dataset_name,
            'model': model_name,
            'balanced_accuracy': bal_acc,
            'f1_score_bad_class': f1
        })
        
    return results_list

Se definen los 4 datasets a probar: German Credit, Taiwan Credit Default, Credit Card Fraud y Give Me Some Credit.

Un paso clave en la carga de datos es **mapear la variable objetivo (y)** para que la **clase minoritaria (impago/fraude) sea siempre 1** y la mayoritaria sea 0. Esto estandariza la evaluación.

A continuación, se ejecuta el benchmark convencional.

In [4]:
# --- 1. Benchmark Convencional (Tarea 1) ---

# Diccionario de datasets (con "Give Me Some Credit" activado)
datasets_to_test = {
    'German Credit': {'source': 'openml', 'id': 31, 'target_col': 'class'},
    'Taiwan Credit Default': {'source': 'openml', 'id': 42477, 'target_col': 'class'},
    'Credit Card Fraud': {'source': 'openml', 'id': 1597, 'target_col': 'class'},
    'Give Me Some Credit': {'source': 'csv', 'path': '../data/cs-training.csv', 'target_col': 'SeriousDlqin2yrs'}
}

# Lista para guardar los resultados convencionales
all_results_conv = []

print("🚀 INICIANDO BENCHMARK (1. CONVENCIONAL)")

for name, info in datasets_to_test.items():
    try:
        print(f"\n--- Cargando y procesando dataset: {name} ---")
        
        if info['source'] == 'openml':
            data = fetch_openml(data_id=info['id'], as_frame=True, parser='auto')
            X = data.data
            y_raw = data.target if 'target' in data else data['class']

            # Mapeamos explícitamente para que la clase MINORITARIA (fraude/default) sea 1
            y_str = y_raw.astype(str)
            counts = y_str.value_counts()

            if len(counts) > 2:
                print(f"Advertencia: El dataset {name} tiene más de 2 clases.")
                # Lógica simple para este caso: usar factorize
                y = pd.Series(pd.factorize(y_raw)[0], index=y_raw.index)
            else:
                minority_label = counts.idxmin()
                majority_label = counts.idxmax()
                
                print(f"Mapeando: '{majority_label}' (Mayoritaria) -> 0, '{minority_label}' (Minoritaria) -> 1")
                y = y_str.map({majority_label: 0, minority_label: 1}).astype(int)
                y.index = y_raw.index # Preservar el índice


        elif info['source'] == 'csv':
            df = pd.read_csv(info['path'])
            # Asumimos que la primera columna (Unnamed: 0) es un índice y la quitamos
            if 'Unnamed: 0' in df.columns:
                df = df.drop(columns=['Unnamed: 0'])
            X = df.drop(columns=[info['target_col']])
            y_raw = df[info['target_col']]
            
            # El dataset CSV tiene valores nulos, los rellenamos con la media (estrategia simple)
            X = X.fillna(X.mean())

            # Este dataset ya es 0 (bueno) y 1 (malo). 
            # No usamos factorize, solo aseguramos que sea 'int'.
            y = y_raw.astype(int)

        
        # Guardar los datos para las siguientes ejecuciones
        info['X_data'] = X
        info['y_data'] = y

        # Llamar a la función de evaluación CONVENCIONAL (celda [5])
        results = evaluate_models(X, y, models_to_test, name)
        all_results_conv.extend(results)
        print(f"Evaluación convencional para {name} completada.")

    except Exception as e:
        print(f"🚨 No se pudo procesar el dataset {name}. Error: {e}")

# Crear el DataFrame de resultados (esto reemplaza tu celda [7])
final_results_conv_df = pd.DataFrame(all_results_conv)
print("\n✅ --- TABLA COMPARATIVA (1. Convencional) --- ✅")
display(final_results_conv_df.sort_values(by=['dataset', 'balanced_accuracy'], ascending=[True, False]))

🚀 INICIANDO BENCHMARK (1. CONVENCIONAL)

--- Cargando y procesando dataset: German Credit ---
Mapeando: 'good' (Mayoritaria) -> 0, 'bad' (Minoritaria) -> 1
--- Evaluando Dataset: German Credit ---

Probando modelo: Logistic Regression...
Matriz de Confusión:
[[187  23]
 [ 42  48]]
Balanced Accuracy: 0.712
F1-Score (clase 'mala'): 0.596

Probando modelo: Random Forest...
Matriz de Confusión:
[[196  14]
 [ 55  35]]
Balanced Accuracy: 0.661
F1-Score (clase 'mala'): 0.504

Probando modelo: Decision Tree...
Matriz de Confusión:
[[158  52]
 [ 51  39]]
Balanced Accuracy: 0.593
F1-Score (clase 'mala'): 0.431
Evaluación convencional para German Credit completada.

--- Cargando y procesando dataset: Taiwan Credit Default ---
Mapeando: '0' (Mayoritaria) -> 0, '1' (Minoritaria) -> 1
--- Evaluando Dataset: Taiwan Credit Default ---

Probando modelo: Logistic Regression...
Matriz de Confusión:
[[6805  204]
 [1521  470]]
Balanced Accuracy: 0.603
F1-Score (clase 'mala'): 0.353

Probando modelo: Random

,dataset,model,balanced_accuracy,f1_score_bad_class
7,Credit Card Fraud,Random Forest,0.881727,0.849624
8,Credit Card Fraud,Decision Tree,0.868091,0.770318
6,Credit Card Fraud,Logistic Regression,0.807345,0.716535
0,German Credit,Logistic Regression,0.711905,0.596273
1,German Credit,Random Forest,0.661111,0.503597
2,German Credit,Decision Tree,0.592857,0.430939
11,Give Me Some Credit,Decision Tree,0.615006,0.276404
10,Give Me Some Credit,Random Forest,0.587051,0.277500
9,Give Me Some Credit,Logistic Regression,0.519659,0.077471
4,Taiwan Credit Default,Random Forest,0.651745,0.461736


### 2.1. Resultados del Benchmark Convencional

La tabla de resultados y la de promedios demuestran el problema central:

* **Fracaso Total en Desbalanceo Extremo:** En "Give Me Some Credit" (6.7% de impagos), la Regresión Logística convencional obtiene un **F1-Score de 0.077**. Esto es un fracaso total: el modelo no tiene ninguna capacidad real de detectar impagos.
* **Precisión Engañosa:** En "Credit Card Fraud" (0.17% de fraude), los modelos convencionales (especialmente RF y DT) obtienen un `Balanced Accuracy` y un `F1-Score` altos (ej. RF F1-Score 0.850). Esto se debe a que el conjunto de prueba es muy pequeño y los modelos "tuvieron suerte", pero esta métrica es engañosa, como veremos.
* **Línea Base Promedio:** En promedio, el Random Forest (Bal. Acc. 0.695) es el mejor modelo convencional.

In [5]:
# Celda 6: Calcular y mostrar el rendimiento medio de cada modelo
print("📊 --- RENDIMIENTO MEDIO POR MODELO CONVENCIONAL (TODOS LOS DATASETS) --- 📊")

# Agrupamos por 'model' y calculamos la media de las métricas
average_performance = final_results_conv_df.groupby('model')[['balanced_accuracy', 'f1_score_bad_class']].mean()

# Ordenamos por la métrica que consideremos más importante para ver el ranking
average_performance_sorted = average_performance.sort_values(by='balanced_accuracy', ascending=False)

display(average_performance_sorted)

📊 --- RENDIMIENTO MEDIO POR MODELO CONVENCIONAL (TODOS LOS DATASETS) --- 📊


,balanced_accuracy,f1_score_bad_class
model,,
Random Forest,0.695409,0.523114
Decision Tree,0.671236,0.467510
Logistic Regression,0.660597,0.435750


## 3. Función de Evaluación 2: Estrategia de Coste Sensible (CSL)

La función `evaluate_cost_sensitive_models` implementa la primera de nuestras estrategias avanzadas: el **Aprendizaje Sensible a Costes (CSL)** en su forma más simple.

**¿Qué hace esta función?**
* Es idéntica a la función convencional, con **una diferencia clave**:
* Al crear el `Pipeline`, re-inicializa cada modelo y le añade explícitamente el parámetro **`class_weight='balanced'`**.

**¿Qué significa `class_weight='balanced'`?**
* Modifica la **función de coste del algoritmo**.
* Le dice al modelo que, durante el entrenamiento (`.fit()`), debe **penalizar más los errores en la clase minoritaria** (impago/fraude) y menos los de la clase mayoritaria.
* El peso de la penalización es inversamente proporcional a la frecuencia de la clase, equilibrando su "importancia".
* Esto es un **cambio a nivel de algoritmo**, no a nivel de datos.
* Los resultados de esta función se usarán para la tabla "Tipo 2: Coste Sensible".

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report # Asegurarse de que classification_report está importado

def evaluate_cost_sensitive_models(X, y, models, dataset_name):
    """
    Versión de la función de evaluación que entrena los modelos
    con sensibilidad al coste y MUESTRA los resultados detallados.
    """
    print(f"--- Evaluando Dataset (Sensible a Costes): {dataset_name} ---")
    
    results_list = []
    
    # Mismo preprocesamiento y división que antes
    categorical_features = X.select_dtypes(include=['category', 'object']).columns
    numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
        ],
        remainder='passthrough'
    )
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    
    for model_name, model in models.items():
        print(f"\nProbando modelo: {model_name} (Sensible al Coste)...")
        
        # Re-inicializamos el modelo con el parámetro class_weight='balanced'
        if model_name == 'Logistic Regression':
            model_cs = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced')
        elif model_name == 'Random Forest':
            model_cs = RandomForestClassifier(random_state=42, class_weight='balanced')
        elif model_name == 'SVC':
            model_cs = SVC(random_state=42, class_weight='balanced')
        elif model_name == 'Decision Tree':
            model_cs = DecisionTreeClassifier(random_state=42, class_weight='balanced')

        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', model_cs)
        ])
        
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        
        # --- CÓDIGO AÑADIDO PARA MOSTRAR DETALLES ---
        print("Matriz de Confusión:")
        print(confusion_matrix(y_test, y_pred))
        
        bal_acc = balanced_accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, pos_label=1, zero_division=0)
        
        print(f"Balanced Accuracy: {bal_acc:.3f}")
        print(f"F1-Score (clase 'mala'): {f1:.3f}")
        print("Reporte de Clasificación:")
        # Usamos try-except por si alguna predicción no tiene ambas clases
        try:
            target_names = [f'Clase {i}' for i in sorted(y.unique())]
            print(classification_report(y_test, y_pred, target_names=target_names, zero_division=0))
        except:
            print(classification_report(y_test, y_pred, zero_division=0))

        # --- FIN DEL CÓDIGO AÑADIDO ---

        results_list.append({
            'dataset': dataset_name,
            'model': model_name,
            'balanced_accuracy': bal_acc,
            'f1_score_bad_class': f1
        })
        
    return results_list

## 4. Función de Evaluación 3: Estrategia de Balanceo (SMOTE)

La función `evaluate_smote_models` implementa la segunda estrategia avanzada: el **Balanceo de Datos** usando la técnica **SMOTE** (Synthetic Minority Over-sampling Technique).

**¿Qué hace esta función?**
* Esta función tiene **dos diferencias clave** con la convencional:
1.  Utiliza `ImbPipeline` (un pipeline de la librería `imblearn`) en lugar del `Pipeline` estándar de `sklearn`.
2.  Añade un nuevo paso al pipeline: `('smote', SMOTE(random_state=42))`.

**¿Cómo funciona?**
* Esto es un **cambio a nivel de datos**.
* Cuando se llama a `.fit()`, `ImbPipeline` primero aplica `SMOTE` **solo a los datos de entrenamiento (`X_train`)**. SMOTE crea "muestras sintéticas" de la clase minoritaria hasta que el dataset de entrenamiento queda balanceado.
* Luego, el modelo (convencional, con `class_weight=None`) se entrena con este **nuevo set de datos balanceado**.
* Es crucial que `SMOTE` no se aplica al `X_test` (para evitar fuga de datos), y el `ImbPipeline` maneja esto automáticamente.
* Los resultados de esta función se usarán para la tabla "Tipo 3: Balanceo (SMOTE)".

In [7]:
# --- 2.b. Función de Evaluación (Balanceo SMOTE) ---

def evaluate_smote_models(X, y, models, dataset_name):
    """
    Versión de la función de evaluación que entrena los modelos
    con balanceo SMOTE (Tarea 3.8).
    """
    print(f"--- Evaluando Dataset (Balanceo SMOTE): {dataset_name} ---")
    
    results_list = []
    
    # Mismo preprocesamiento y división
    categorical_features = X.select_dtypes(include=['category', 'object']).columns
    numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
        ],
        remainder='passthrough'
    )
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    
    for model_name, model in models.items():
        
        # 1. Crear el pipeline de IMBLearn (¡Importante!)
        # SMOTE solo se aplica al .fit(), no al .predict()
        pipeline_smote = ImbPipeline(steps=[
            ('preprocessor', preprocessor),
            ('smote', SMOTE(random_state=42)), # 2. Añadir SMOTE
            ('classifier', model.set_params(class_weight=None)) # 3. Usar el modelo *convencional*
        ])
        
        # Entrenar
        pipeline_smote.fit(X_train, y_train)
        y_pred = pipeline_smote.predict(X_test)
        
        # Calcular métricas
        bal_acc = balanced_accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, pos_label=1, zero_division=0) # Asumimos 1 es la clase 'mala'

        results_list.append({
            'dataset': dataset_name,
            'model': model_name,
            'balanced_accuracy': bal_acc,
            'f1_score_bad_class': f1,
            'Tipo': '3. Balanceo (SMOTE)' # Añadimos el tipo
        })
        
    return results_list

In [8]:
# --- 2. Benchmark (Coste Sensible 'balanced') ---

all_results_cs = []

print("\n🚀 INICIANDO BENCHMARK (2. COSTE SENSIBLE)")

# Este bucle ahora REUTILIZA los datos X e y cargados en la celda [6]
for name, info in datasets_to_test.items():
    try:
        if 'X_data' in info: # Solo si se cargó correctamente antes
            print(f"\nProcesando {name} (Coste Sensible)...")
            
            # Usamos los datos limpios y guardados
            X = info['X_data']
            y = info['y_data']
            
            # Llamar a la función de evaluación de COSTE SENSIBLE (celda [9])
            results_cs = evaluate_cost_sensitive_models(X, y, models_to_test, name)
            all_results_cs.extend(results_cs)
            print(f"Evaluación CSL para {name} completada.")
        
        else:
            print(f"Saltando {name} (no se cargaron los datos en el paso anterior).")

    except Exception as e:
        print(f"🚨 No se pudo procesar el dataset {name} (CSL). Error: {e}")

final_results_cs_df = pd.DataFrame(all_results_cs)
print("\n✅ --- TABLA COMPARATIVA (2. Coste Sensible) --- ✅")
display(final_results_cs_df.sort_values(by=['dataset', 'balanced_accuracy'], ascending=[True, False]))


🚀 INICIANDO BENCHMARK (2. COSTE SENSIBLE)

Procesando German Credit (Coste Sensible)...
--- Evaluando Dataset (Sensible a Costes): German Credit ---

Probando modelo: Logistic Regression (Sensible al Coste)...
Matriz de Confusión:
[[150  60]
 [ 21  69]]
Balanced Accuracy: 0.740
F1-Score (clase 'mala'): 0.630
Reporte de Clasificación:
              precision    recall  f1-score   support

     Clase 0       0.88      0.71      0.79       210
     Clase 1       0.53      0.77      0.63        90

    accuracy                           0.73       300
   macro avg       0.71      0.74      0.71       300
weighted avg       0.77      0.73      0.74       300


Probando modelo: Random Forest (Sensible al Coste)...
Matriz de Confusión:
[[194  16]
 [ 61  29]]
Balanced Accuracy: 0.623
F1-Score (clase 'mala'): 0.430
Reporte de Clasificación:
              precision    recall  f1-score   support

     Clase 0       0.76      0.92      0.83       210
     Clase 1       0.64      0.32      0.43   

,dataset,model,balanced_accuracy,f1_score_bad_class
6,Credit Card Fraud,Logistic Regression,0.924743,0.119169
7,Credit Card Fraud,Random Forest,0.851334,0.815686
8,Credit Card Fraud,Decision Tree,0.824137,0.695652
0,German Credit,Logistic Regression,0.740476,0.630137
1,German Credit,Random Forest,0.623016,0.429630
2,German Credit,Decision Tree,0.591270,0.432432
9,Give Me Some Credit,Logistic Regression,0.721863,0.282618
11,Give Me Some Credit,Decision Tree,0.598272,0.252240
10,Give Me Some Credit,Random Forest,0.572509,0.241678
3,Taiwan Credit Default,Logistic Regression,0.663928,0.467838


### 3.1. Resultados del Benchmark CSL (`class_weight='balanced'`)

Los resultados de CSL revelan un patrón fundamental:

* **Mejora en Desbalanceo Moderado:** En "Give Me Some Credit" (6.7%), el CSL mejora drásticamente el F1-Score de la Regresión Logística (de 0.077 a **0.283**).
* **Fracaso en Desbalanceo Extremo:** En "Credit Card Fraud" (0.17%), el CSL (`'balanced'`) es **desastroso**. El F1-Score de la Regresión Logística colapsa de 0.717 (convencional) a **0.119**.
* **Fallo en Random Forest:** La `Balanced Accuracy` promedio de RF *empeora* (de 0.695 a 0.673).

**Conclusión clave:** `class_weight='balanced'` funciona para modelos lineales en desbalanceo *moderado*, pero **falla estrepitosamente** en desbalanceo *extremo* (colapsa el F1-Score) y no funciona bien con Random Forest.

In [32]:
# Celda 6: Calcular y mostrar el rendimiento medio de cada modelo
print("📊 --- RENDIMIENTO MEDIO POR MODELO COSTE (TODOS LOS DATASETS) --- 📊")

# Agrupamos por 'model' y calculamos la media de las métricas
average_performance = final_results_cs_df.groupby('model')[['balanced_accuracy', 'f1_score_bad_class']].mean()

# Ordenamos por la métrica que consideremos más importante para ver el ranking
average_performance_sorted = average_performance.sort_values(by='balanced_accuracy', ascending=False)

display(average_performance_sorted)

📊 --- RENDIMIENTO MEDIO POR MODELO COSTE (TODOS LOS DATASETS) --- 📊


,balanced_accuracy,f1_score_bad_class
model,,
Logistic Regression,0.762753,0.374941
Random Forest,0.673069,0.484077
Decision Tree,0.657942,0.446363


In [33]:
# --- 3. Benchmark (Balanceo SMOTE) ---

all_results_smote = []

print("\n🚀 INICIANDO BENCHMARK (3. BALANCEO SMOTE)")

for name, info in datasets_to_test.items():
    try:
        if 'X_data' in info: 
            print(f"\nProcesando {name} (SMOTE)...")
            X = info['X_data']
            y = info['y_data']
            
            results_smote = evaluate_smote_models(X, y, models_to_test, name)
            all_results_smote.extend(results_smote)
            print(f"Evaluación SMOTE para {name} completada.")
        
        else:
            print(f"Saltando {name} (no se cargaron los datos).")
            
    except Exception as e:
        print(f"🚨 No se pudo procesar el dataset {name} (SMOTE). Error: {e}")

final_results_smote_df = pd.DataFrame(all_results_smote)
print("\n✅ --- TABLA COMPARATIVA (3. Balanceo SMOTE) --- ✅")
display(final_results_smote_df.sort_values(by=['dataset', 'balanced_accuracy'], ascending=[True, False]))


🚀 INICIANDO BENCHMARK (3. BALANCEO SMOTE)

Procesando German Credit (SMOTE)...
--- Evaluando Dataset (Balanceo SMOTE): German Credit ---
Evaluación SMOTE para German Credit completada.

Procesando Taiwan Credit Default (SMOTE)...
--- Evaluando Dataset (Balanceo SMOTE): Taiwan Credit Default ---
Evaluación SMOTE para Taiwan Credit Default completada.

Procesando Credit Card Fraud (SMOTE)...
--- Evaluando Dataset (Balanceo SMOTE): Credit Card Fraud ---
Evaluación SMOTE para Credit Card Fraud completada.

Procesando Give Me Some Credit (SMOTE)...
--- Evaluando Dataset (Balanceo SMOTE): Give Me Some Credit ---
Evaluación SMOTE para Give Me Some Credit completada.

✅ --- TABLA COMPARATIVA (3. Balanceo SMOTE) --- ✅


,dataset,model,balanced_accuracy,f1_score_bad_class,Tipo
6,Credit Card Fraud,Logistic Regression,0.930873,0.115215,3. Balanceo (SMOTE)
7,Credit Card Fraud,Random Forest,0.891804,0.831541,3. Balanceo (SMOTE)
8,Credit Card Fraud,Decision Tree,0.873927,0.502262,3. Balanceo (SMOTE)
0,German Credit,Logistic Regression,0.740476,0.631579,3. Balanceo (SMOTE)
1,German Credit,Random Forest,0.667460,0.522876,3. Balanceo (SMOTE)
2,German Credit,Decision Tree,0.580952,0.409091,3. Balanceo (SMOTE)
9,Give Me Some Credit,Logistic Regression,0.716450,0.268322,3. Balanceo (SMOTE)
10,Give Me Some Credit,Random Forest,0.648427,0.353884,3. Balanceo (SMOTE)
11,Give Me Some Credit,Decision Tree,0.622387,0.250965,3. Balanceo (SMOTE)
4,Taiwan Credit Default,Random Forest,0.675323,0.498507,3. Balanceo (SMOTE)


### 4.1. Resultados del Benchmark SMOTE

Los resultados de SMOTE (visibles en la tabla de la celda 13) muestran un patrón diferente:

* **Éxito en Regresión Logística:** Al igual que CSL, SMOTE **mejora drásticamente** el rendimiento de LR (promedio de 0.661 a 0.762).
* **Éxito en Random Forest:** A diferencia de CSL, SMOTE **sí mejora** el rendimiento de RF (promedio de 0.695 a 0.720).

Esto sugiere que los modelos de Random Forest se benefician más de ver un conjunto de datos balanceado (SMOTE) que de una penalización interna (CSL).

In [ ]:
# --- TABLA COMPARATIVA FINAL (TAREA 7) ---

# Añadimos una columna 'Tipo' para diferenciar los resultados
final_results_conv_df['Tipo'] = '1. Convencional'
final_results_cs_df['Tipo'] = '2. Coste Sensible (class_weight)'
final_results_smote_df['Tipo'] = '3. Balanceo (SMOTE)' 

# Unimos las TRES tablas de resultados
comparison_df = pd.concat([final_results_conv_df, final_results_cs_df, final_results_smote_df]) 

# Mostramos la tabla final, ordenada para facilitar la comparación
print("📊 --- TABLA COMPARATIVA FINAL: ESTRATEGIAS vs. DATASETS --- 📊")
display(comparison_df.sort_values(by=['dataset', 'model', 'Tipo']))

📊 --- TABLA COMPARATIVA FINAL: ESTRATEGIAS vs. DATASETS --- 📊


,dataset,model,balanced_accuracy,f1_score_bad_class,Tipo
8,Credit Card Fraud,Decision Tree,0.868091,0.770318,1. Convencional
8,Credit Card Fraud,Decision Tree,0.824137,0.695652,2. Coste Sensible (class_weight)
8,Credit Card Fraud,Decision Tree,0.873927,0.502262,3. Balanceo (SMOTE)
6,Credit Card Fraud,Logistic Regression,0.807345,0.716535,1. Convencional
6,Credit Card Fraud,Logistic Regression,0.924743,0.119169,2. Coste Sensible (class_weight)
6,Credit Card Fraud,Logistic Regression,0.930873,0.115215,3. Balanceo (SMOTE)
7,Credit Card Fraud,Random Forest,0.881727,0.849624,1. Convencional
7,Credit Card Fraud,Random Forest,0.851334,0.815686,2. Coste Sensible (class_weight)
7,Credit Card Fraud,Random Forest,0.891804,0.831541,3. Balanceo (SMOTE)
2,German Credit,Decision Tree,0.592857,0.430939,1. Convencional


## 5. Conclusiones Finales del Benchmark

La tabla comparativa final consolida los hallazgos:

1.  **Para Regresión Logística (Lineal):**
    * En desbalanceo **moderado** ("German Credit", "Taiwan"), tanto **CSL (`'balanced'`)** como **SMOTE** son estrategias excelentes y muy similares, mejorando tanto `Balanced Accuracy` como `F1-Score` (ej. German F1-Score: 0.596 -> **0.630**).
    * En desbalanceo **extremo** ("Credit Card Fraud"), ambas estrategias **fallan**, colapsando el `F1-Score` (0.717 -> **0.119** con CSL y **0.115** con SMOTE).

2.  **Para Random Forest (Bagging):**
    * **SMOTE** es la estrategia ganadora en promedio (mejora F1 de 0.523 a 0.552).
    * **CSL (`'balanced'`)** es una mala estrategia; empeora tanto la `Balanced Accuracy` como el `F1-Score` en promedio.

**Conclusión General:** Este notebook justifica la necesidad de estrategias más avanzadas. `class_weight='balanced'` y `SMOTE` (con parámetros por defecto) no son soluciones universales y **fallan en los casos de desbalanceo extremo**, que es donde se centra el notebook `modelos_avanzados.ipynb`.